# **Análise de Sentimentos de _Reviews_ de produtos na plataforma Steam**

## **Importando bibliotecas**

In [1]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np 
import re
import string
import unidecode

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/juan-
[nltk_data]     burtet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## **Carregando _dataset_** 

In [3]:
data = pd.read_csv('dataset/steam_reviews.tsv', sep='\t')[['review', 'voted_up']]

In [4]:
data.describe()

,review,voted_up
count,507051,507052
unique,506693,2
top,Err:507,True
freq,226,460579


In [5]:
data.dtypes

review      object
voted_up      bool
dtype: object

In [6]:
data.head(5)

,review,voted_up
0,"Corrupted é um jogo de RPG bem simplificado, o...",True
1,Meu PC tem uma placa de vídeo equivalente a GT...,False
2,Muito Bom! O meu diz que é recomendado specs:...,True
3,Funciona,True
4,"Muito bom o Steam VR, Very good.",True


### **Separando Exemplos**

#### Selecionando apenas Exemplos Positivos

In [7]:
positive = data[data['voted_up'] == True].sample(46000)
positive.head(5)

,review,voted_up
415759,De longe um dos melhores jogos de sobrevivênci...,True
433841,"Muito f0da, incrível pra caralh#",True
17994,"Esse Jogo é Incrivel, um dos melhores jogos qu...",True
333474,e bom para passar o tempo mais o multiplayer e...,True
333443,"Divertido, o ruim é que o Co-op é apenas local...",True


#### Selecionando apenas Exemplos Negativos

In [8]:
negative = data[data['voted_up'] == False].sample(46000)
negative.head(5)

,review,voted_up
180320,"Horrível, gráficos péssimos, jogabilidade horr...",False
138632,Não dá pra jogar. Dublagem terrível e mecânica...,False
423561,"Tentei mesmo gostar deste jogo, mas infelizmen...",False
335857,BLOCKADE 3D É um jogo otimo para quem e fan de...,False
159700,muita plataforma pra pular e pouca história e ...,False


#### Concatenando os dados e misturando-os

In [9]:
frames = [positive, negative]
data = pd.concat(frames)
data = data.sample(frac=1).reset_index(drop=True)
data.head(10)

,review,voted_up
0,"Jogo otimo,divertido e difícil(para mim melhor...",True
1,"Excelente jogo, já gostava muito antes, mas co...",True
2,bosta total a tela e toda borrada vai a merda,False
3,"Um ótimo jogo, muito bom, ainda mais de coop a...",True
4,Envelheceu muito mal e não tem nada realmente ...,False
5,"Muito bom, muita coisa a se fazer agrada difer...",True
6,Odallus é um excelente jogo que engloba elemen...,True
7,Jogo Otimo Pra jogar,True
8,"Um bom jogo com bons graficos e boa historia, ...",False
9,"Os poderes são bem divertidos (no início), mas...",False


## **Processando os Dados**

In [19]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('portuguese')).union(stopwords.words('english')).union(["pra"]).union(["são"])

# Function that removes stopwords
def remove_stopwords(text):
    text = text.split(' ')
    text = [x for x in text if x not in stop_words]
    return ' '.join(text)

# Function that removes words with less than 3 chars
def remove_words_less_than_2(text):
    text = text.split(' ')
    text = [x for x in text if len(x) > 2]
    return ' '.join(text)

def process_data(df):
    
    # To lowercase
    df['review'] = df['review'].str.lower()
    
    # Removing punctuations
    df['review'] = df['review'].str.replace('[^\w\s]',' ')
    
    # Remove a single character from the string
    remove_single_char = lambda x: re.sub(r"\b[a-zA-Z]\b", "", x)
    df['review'] = df['review'].apply(remove_single_char)
    
    # Substituting multiple spaces with single space
    remove_multiple_spaces = lambda x: ' '.join(x.split())
    df['review'] = df['review'].apply(remove_multiple_spaces)
    
    # Remove repeated letters if there 3 or more
    remove_repeated_letters = lambda x: re.sub(r'([a-z])\1+', r'\1\1', x)
    df['review'] = df['review'].apply(remove_repeated_letters)
    
    # removing stopwords
    rm_stopwords = lambda x: remove_stopwords(x)
    df['review'] = df['review'].apply(rm_stopwords)
    
    # Removing accents
    remove_accent = lambda x: unidecode.unidecode(x)
    df['review'] = df['review'].apply(remove_accent)
    
    # Remove words with less than 3 chars
    remove_less_2 = lambda x: remove_words_less_than_2(x)
    df['review'] = df['review'].apply(remove_less_2)
    
    return df

In [20]:
data = process_data(data)
data.head(10)

,review,voted_up
0,jogo otimo divertido dificil mim melhor ainda ...,True
1,excelente jogo gostava antes conseguiram melho...,True
2,bosta total tela toda borrada vai merda,False
3,otimo jogo bom ainda coop apesar ser pouco pesado,True
4,envelheceu mal nada realmente importante nesse...,False
5,bom muita coisa fazer agrada diferentes tipos ...,True
6,odallus excelente jogo engloba elementos aprec...,True
7,jogo otimo jogar,True
8,bom jogo bons graficos boa historia joguei pou...,False
9,poderes bem divertidos inicio estoria jogo emp...,False


# TF-IDF

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words = set(stopwords.words('portuguese')).union(set(stopwords.words('english')))

corpus = data['review'].values

tfidf = TfidfVectorizer(stop_words=stop_words)
response = tfidf.fit_transform(corpus)

scores = zip(tfidf.get_feature_names(), np.asarray(response.sum(axis=0)).ravel())
sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
sorted_scores[:25]

[('jogo', 5983.393129911418),
 ('bom', 2932.6897207824145),
 ('jogar', 1745.2510137470601),
 ('recomendo', 1686.395342613354),
 ('game', 1510.7186433537838),
 ('bem', 1475.5469723054327),
 ('melhor', 1429.2816436437633),
 ('nao', 1307.281568156953),
 ('otimo', 1155.7941290580222),
 ('historia', 1145.754567640435),
 ('divertido', 1055.8616563902629),
 ('ruim', 1031.2171625126562),
 ('ser', 1025.9703557629427),
 ('jogabilidade', 1009.9797782768869),
 ('vale', 976.768124333627),
 ('tempo', 976.5032208555782),
 ('jogos', 915.6021793828961),
 ('legal', 906.7505375260512),
 ('pena', 890.1663408014688),
 ('graficos', 861.1558113307284),
 ('ainda', 836.3853118683086),
 ('joguei', 825.1352969322043),
 ('gostei', 775.286453822748),
 ('amigos', 769.6533217583781),
 ('boa', 750.292459882427)]

### Calcula a media e desvio padrão dos scores do TF-IDF

In [22]:
import statistics as st

scores_labels = [x[1] for x in sorted_scores]
media = sum(scores_labels)/len(scores_labels)
dp = st.stdev(scores_labels)

### Pega apenas os scores com valores acima da média + 1 desvio padrão

In [23]:
best_scores = []
for score in sorted_scores:
    if score[1] < media + dp:
        break
    best_scores.append(score)

### Diferença no tamanho dos conjuntos

In [24]:
print("Tamanho do conjunto original:", len(sorted_scores))
print("Tamanho do conjunto com os melhores scores:", len(best_scores))

Tamanho do conjunto original: 78604
Tamanho do conjunto com os melhores scores: 1320


## Criação do dataset utilizado no modelo de aprendizado

### Testando as melhores combinações de entrada

In [40]:
# Corpus do texto
corpus = data['review'].values
size = int(len(best_scores)/3)

# Usado para retornar o corpus apenas com as palavras top
def refined_corpus(label_scores, corpus):
    only_words = set([x[0] for x in label_scores])

    new_corpus = []
    for text in corpus:
        words = text.split(" ")
        new_text = [word for word in words if word in only_words]
        new_corpus.append(' '.join(new_text))
    
    return new_corpus

### Testando com o primeiro, segundo e terceiro terço

In [51]:
# Primeiro terço
begin = 0
end = begin + size
input_scores = best_scores[begin:end]
corpus_1_3 = refined_corpus(input_scores, corpus)

# Segundo terço
begin = size
end = begin + size
input_scores = best_scores[begin:end]
corpus_2_3 = refined_corpus(input_scores, corpus)

# Terceiro terço
begin = size * 2
end = begin + size
input_scores = best_scores[begin:end]
corpus_3_3 = refined_corpus(input_scores, corpus)

# Utilizando todos os melhores
corpus_best = refined_corpus(best_scores, corpus)

list_corpus = [
    corpus_1_3,
    corpus_2_3,
    corpus_3_3,
    corpus_best,
]


In [52]:
qts = []
for c in list_corpus:
    tfidf = TfidfVectorizer(stop_words=stop_words)
    response = tfidf.fit_transform(c)
    
    total = 0
    for review in response.toarray():
        x = sum(review)
        if x > 0:
            total += 1
    
    qts.append(total)

In [54]:
print("Corpus 1:", qts[0]/len(list_corpus[0]))
print("Corpus 2:", qts[1]/len(list_corpus[0]))
print("Corpus 3:", qts[2]/len(list_corpus[0]))
print("Full Corpus:", qts[3]/len(list_corpus[0]))

Corpus 1: 0.9341086956521739
Corpus 2: 0.6161630434782609
Corpus 3: 0.472695652173913
Full Corpus: 0.9579673913043478


### Utilizar o primeiro terço, pois engobla 93% do dataset original

In [56]:
from sklearn.model_selection import train_test_split

tfidf = TfidfVectorizer(stop_words=stop_words)

processed_features = tfidf.fit_transform(corpus_1_3).toarray()
labels = data['voted_up'].values

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

In [57]:
from sklearn.naive_bayes import MultinomialNB

nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)
predicted = nb_clf.predict(X_test)
np.mean(predicted == y_test)

0.7970652173913043

In [58]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,predicted))
print(accuracy_score(y_test, predicted))

[[6899 2309]
 [1425 7767]]
              precision    recall  f1-score   support

       False       0.83      0.75      0.79      9208
        True       0.77      0.84      0.81      9192

   micro avg       0.80      0.80      0.80     18400
   macro avg       0.80      0.80      0.80     18400
weighted avg       0.80      0.80      0.80     18400

0.7970652173913043
